# Лекция №6

В данном уроке мы познакомимся с тем, как использваоть предопученные нейронные сети с помощью OpenCV

Сегодняшнее сообщение в блоге вдохновлено несколькими читателями PyImageSearch, которые прокомментировали предыдущие уроки глубокого обучения, желая понять, что именно делает функция OpenCV blobFromImage под капотом.

Видите ли, чтобы получить (правильные) прогнозы из глубоких нейронных сетей, вам сначала нужно предварительно обработать ваши данные.

В контексте глубокого обучения и классификации изображений эти задачи предварительной обработки обычно включают в себя:

Среднее вычитание
Масштабирование по некоторому фактору
Новый модуль OpenCV для глубокой нейронной сети (dnn) содержит две функции, которые можно использовать для предварительной обработки изображений и подготовки их к классификации с помощью предварительно обученных моделей глубокого обучения.

В сегодняшнем посте мы разберем функции предварительной обработки OpenCV cv2.dnn.blobFromImage и cv2.dnn.blobFromImages и поймем, как они работают.

Чтобы узнать больше о предварительной обработке изображений для глубокого обучения через OpenCV, просто продолжайте читать.

Глубокое изучение: как работает blobFromImage в OpenCV
OpenCV предоставляет две функции, облегчающие предварительную обработку изображений для глубокой классификации обучения:

cv2.dnn.blobFromImage
cv2.dnn.blobFromImages
Эти две функции выполняют

Среднее вычитание
пересчет
И при желании обмен каналов
В оставшейся части этого урока мы будем:

Изучите среднее вычитание и масштабирование
Изучите сигнатуру функции каждой функции предварительной обработки глубокого обучения
Изучите эти методы подробно
И, наконец, примените функции глубокого обучения OpenCV к набору входных изображений.
Давайте идти вперед и начать.

Глубокое обучение и среднее вычитание

<img src="img/blob_from_images_mean_subtraction.jpg" alt="Drawing" style="width: 600px;"/>
Рисунок 1: Визуальное представление среднего вычитания, где среднее значение RGB (в центре) было вычислено из набора данных изображений и вычтено из исходного изображения (слева), что привело к выходному изображению (справа).

Прежде чем мы углубимся в объяснение функций предварительной обработки глубокого обучения OpenCV, нам сначала необходимо понять среднее вычитание. Среднее вычитание используется для борьбы с изменениями освещенности входных изображений в нашем наборе данных. Поэтому мы можем рассматривать среднее вычитание как метод, используемый для помощи нашим сверточным нейронным сетям.

Прежде чем мы начнем тренировать нашу глубокую нейронную сеть, мы сначала вычисляем среднюю интенсивность пикселей по всем изображениям в обучающем наборе для каждого из красного, зеленого и синего каналов.

Это означает, что в итоге мы получим три переменные:\mu_R, \mu_G, and \mu_B


Обычно результирующие значения представляют собой 3-х кортеж, состоящий из среднего значения красного, зеленого и синего каналов соответственно.

Например, средними значениями для обучающего набора ImageNet являются R = 103,93, G = 116,77 и B = 123,68 (вы, возможно, уже сталкивались с этими значениями ранее, если использовали сеть, которая была предварительно обучена в ImageNet).

Однако в некоторых случаях средние значения красного, зеленого и синего могут быть вычислены по каналам, а не по пикселям, что приводит к матрице MxN. В этом случае матрица MxN для каждого канала затем вычитается из входного изображения во время обучения / тестирования.

Оба метода являются совершенно правильными формами среднего вычитания; однако, мы склонны видеть, что пиксельная версия используется чаще, особенно для больших наборов данных.

Когда мы готовы передать изображение через нашу сеть (для обучения или тестирования), мы вычитаем среднее значение \ mu из каждого входного канала входного изображения:

R = R - \mu_R
G = G - \mu_G
B = B - \mu_B
У нас также может быть коэффициент масштабирования \ sigma, который добавляет нормализацию:

R = (R - \mu_R) / \sigma

G = (G - \mu_G) / \sigma

B = (B - \mu_B) / \sigma

Значение \ sigma может быть стандартным отклонением по обучающему набору (тем самым превращая шаг предварительной обработки в стандартную оценку / z-оценку). Тем не менее, \ sigma также может быть установлен вручную (по сравнению с вычисленным), чтобы масштабировать пространство входного изображения в конкретный диапазон - это действительно зависит от архитектуры, от того, как была обучена сеть, и от тех методов, с которыми знаком разработчик.

Важно отметить, что не все архитектуры с глубоким обучением выполняют среднее вычитание и масштабирование! Перед предварительной обработкой изображений обязательно прочитайте соответствующую публикацию / документацию по используемой вами глубокой нейронной сети.

Как вы обнаружите в своем пути глубокого обучения, некоторые архитектуры выполняют только среднее вычитание (тем самым устанавливая \ sigma = 1). Другие архитектуры выполняют как среднее вычитание, так и масштабирование. Даже другие архитектуры предпочитают не вычитать или масштабировать. Всегда проверяйте соответствующую публикацию, которую вы внедряете / используете, чтобы проверить методы, которые использует автор.


Функции OpenCV blobFromImage и blobFromImages
Начнем с обращения к официальной документации OpenCV для cv2.dnn.blobFromImage:

[blobFromImage] создает 4-мерное пятно из изображения. Опционально изменяет размеры и обрезает изображение по центру, вычитает средние значения, масштабирует значения по коэффициенту масштабирования, меняет синий и красный каналы.

Неформально, BLOB-объект - это просто (потенциально коллекция) изображений с одинаковыми пространственными размерами (то есть, шириной и высотой), одинаковой глубиной (числом каналов), которые должны быть предварительно обработаны одинаковым образом.

Функции cv2.dnn.blobFromImage и cv2.dnn.blobFromImages практически идентичны.

Давайте начнем с проверки подписи функции cv2.dnn.blobFromImage ниже:

blob = cv2.dnn.blobFromImage (изображение, коэффициент масштабирования = 1,0, размер, среднее значение, swapRB = True)

Я представил обсуждение каждого параметра ниже:

изображение: это входное изображение, которое мы хотим предварительно обработать, прежде чем передать его через нашу глубокую нейронную сеть для классификации.
Коэффициент масштабирования: после выполнения среднего вычитания мы можем по желанию масштабировать изображения. По умолчанию это значение равно 1,0 (то есть без масштабирования), но мы также можем указать другое значение. Также важно отметить, что коэффициент масштабирования должен быть равен 1 / \ sigma, поскольку мы фактически умножаем входные каналы (после среднего вычитания) на коэффициент масштабирования.
размер: здесь мы предоставляем пространственный размер, который ожидает сверточная нейронная сеть. Для большинства современных нейронных сетей это 224 × 224, 227 × 227 или 299 × 299.
означает: это наши средние значения вычитания. Они могут быть 3-мя кортежами RGB-средств или могут быть одним значением, и в этом случае предоставленное значение вычитается из каждого канала изображения. Если вы выполняете среднее вычитание, убедитесь, что вы поставили 3-кортеж в порядке (R, G, B), особенно при использовании поведения swapRB = True по умолчанию.
swapRB: OpenCV предполагает, что изображения имеют порядок каналов BGR; однако среднее значение предполагает, что мы используем порядок RGB. Чтобы устранить это расхождение, мы можем поменять местами каналы R и B на изображении, установив для этого значения значение True. По умолчанию OpenCV выполняет этот обмен каналов для нас.
Функция cv2.dnn.blobFromImage возвращает BLOB-объект, который является нашим входным изображением после среднего вычитания, нормализации и переключения каналов.

Функция cv2.dnn.blobFromImages точно такая же:

blob = cv2.dnn.blobFromImages (изображения, коэффициент масштабирования = 1,0, размер, среднее значение, swapRB = True)

Единственное исключение состоит в том, что мы можем передавать несколько изображений, что позволяет нам пакетно обрабатывать набор изображений.

Если вы обрабатываете несколько изображений / кадров, обязательно используйте функцию cv2.dnn.blobFromImages, так как при этом меньше затрат на вызов функции и вы сможете быстрее обрабатывать изображения / кадры.

Глубокое обучение с функцией OpenCV blobFromImage
Теперь, когда мы изучили функции blobFromImage и blobFromImages, давайте применим их к нескольким образцам изображений, а затем передадим их через сверточную нейронную сеть для классификации.

В качестве предварительного условия вам необходима версия OpenCV 3.3.0 как минимум. NumPy - это зависимость от привязок Python в OpenCV, а imutils - мой пакет удобных функций, доступных на GitHub и в индексе пакетов Python.

Если вы не установили OpenCV, вам нужно следовать последним учебным пособиям, доступным здесь, и обязательно указывать OpenCV 3.3.0 или более позднюю версию при клонировании / загрузке opencv и opencv_contrib.

Пакет imutils можно установить через pip:

In [ ]:
$ pip install imutils

Assuming your image processing environment is ready to go, let’s open up a new file, name it blob_from_images.py , and insert the following code:

In [ ]:
# import the necessary packages
from imutils import paths
import numpy as np
import cv2
 
# load the class labels from disk
rows = open("synset_words.txt").read().strip().split("\n")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]
 
# load our serialized model from disk
net = cv2.dnn.readNetFromCaffe("bvlc_googlenet.prototxt",
	"bvlc_googlenet.caffemodel")
 
# grab the paths to the input images
imagePaths = sorted(list(paths.list_images("images/")))

First we import imutils , numpy , and cv2  (Lines 2-4).

Then we read synset_words.txt  (the ImageNet Class labels) and extract classes , our class labels, on Lines 7 and 8.

To load our model model from disk we use the DNN function,  cv2.dnn.readNetFromCaffe , and specify bvlc_googlenet.prototxt  as the filename parameter and bvlc_googlenet.caffemodel  as the actual model file (Lines 11 and 12).

Note: You can grab the pre-trained Convolutional Neural Network, class labels text file, source code, and example images to this post using the “Downloads” section at the bottom of this tutorial.

Finally, we grab the paths to the input images on Line 15. If you’re using Windows you should change the path separator here to ensure you can correctly load the image paths.

Next, we’ll load images from disk and pre-process them using blobFromImage :

In [ ]:
# (1) load the first image from disk, (2) pre-process it by resizing
# it to 224x224 pixels, and (3) construct a blob that can be passed
# through the pre-trained network
image = cv2.imread(imagePaths[0])
resized = cv2.resize(image, (224, 224))
blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
print("First Blob: {}".format(blob.shape))

In this block, we first load the image  (Line 20) and then resize it to 224×224 (Line 21), the required input image dimensions for GoogLeNet.

Now we’re to the crux of this post.

On Line 22, we call cv2.dnn.blobFromImage  which, as stated in the previous section, will create a 4-dimensional blob  for use in our neural net.

Let’s print the shape of our blob so we can analyze it in the terminal later (Line 23).

Next, we’ll feed blob  through GoogLeNet:

In [ ]:
# set the input to the pre-trained deep learning network and obtain
# the output predicted probabilities for each of the 1,000 ImageNet
# classes
net.setInput(blob)
preds = net.forward()
 
# sort the probabilities (in descending) order, grab the index of the
# top predicted label, and draw it on the input image
idx = np.argsort(preds[0])[::-1][0]
text = "Label: {}, {:.2f}%".format(classes[idx],
	preds[0][idx] * 100)
cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX,
	0.7, (0, 0, 255), 2)
 
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

If you’re familiar with recent deep learning posts on this blog, the above lines should look familiar.

We feed the blob  through the network (Lines 28 and 29) and grab the predictions, preds .

Then we sort preds  (Line 33) with the most confident predictions at the front of the list, and generate a label text to display on the image. The label text consists of the class label and the prediction percentage value for the top prediction (Lines 34 and 35).

From there,  we write the label text  at the top of the image  (Lines 36 and 37) followed by displaying the image  on the screen and waiting for a keypress before moving on (Lines 40 and 41).

Now it’s time to use the plural form of the blobFromImage  function.

Here we’ll do (nearly) the same thing, except we’ll instead create and populate a list of images  followed by passing the list as a parameter to blobFromImages :

In [ ]:
# initialize the list of images we'll be passing through the network
images = []
 
# loop over the input images (excluding the first one since we
# already classified it), pre-process each image, and update the
# `images` list
for p in imagePaths[1:]:
	image = cv2.imread(p)
	image = cv2.resize(image, (224, 224))
	images.append(image)
 
# convert the images list into an OpenCV-compatible blob
blob = cv2.dnn.blobFromImages(images, 1, (224, 224), (104, 117, 123))
print("Second Blob: {}".format(blob.shape))

First we initialize our images  list (Line 44), and then, using the imagePaths , we read, resize, and append the image  to the list (Lines 49-52).

Using list slicing, we’ve omitted the first image from imagePaths  on Line 49.

From there, we pass the images  into cv2.dnn.blobFromImages  as the first parameter on Line 55. All other parameters to cv2.dnn.blobFromImages  are identical to cv2.dnn.blobFromImage  above.

For analysis later we print blob.shape  on Line 56.

We’ll next pass the blob  through GoogLeNet and write the class label and prediction at the top of each image:

In [ ]:
# set the input to our pre-trained network and obtain the output
# class label predictions
net.setInput(blob)
preds = net.forward()
 
# loop over the input images
for (i, p) in enumerate(imagePaths[1:]):
	# load the image from disk
	image = cv2.imread(p)
 
	# find the top class label from the `preds` list and draw it on
	# the image
	idx = np.argsort(preds[i])[::-1][0]
	text = "Label: {}, {:.2f}%".format(classes[idx],
		preds[i][idx] * 100)
	cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX,
		0.7, (0, 0, 255), 2)
 
	# display the output image
	cv2.imshow("Image", image)
	cv2.waitKey(0)

The remaining code is essentially the same as above, only our for  loop now handles looping through each of the imagePaths  (again, omitting the first one as we have already classified it).

And that’s it! Let’s see the script in action in the next section.

Summary
In today’s tutorial we examined OpenCV’s blobFromImage  and blobFromImages  deep learning functions.

These methods are used to prepare input images for classification via pre-trained deep learning models.

Both blobFromImage  and blobFromImages  perform mean subtraction and scaling. We can also swap the Red and Blue channels of the image depending on channel ordering. Nearly all state-of-the-art deep learning models perform mean subtraction and scaling — the benefit here is that OpenCV makes these preprocessing tasks dead simple.